In [1]:
import paddle.v2 as paddle
import paddle.v2.dataset.uci_housing as uci_housing
import numpy as np

type(uci_housing.train())
uci_housing.train()
n = 0
for temp in uci_housing.train()():
    if n < 5:
        pass
        #print(temp)
    else:
        n += 1
        

## 测试读取数据
从本地的 my_train_data.txt 中读取数据


In [2]:
train_f = open('my_train_data.txt', 'r')

train_list = train_f.readlines(10)

train_f.close()


#定义数据
x_cycle = []
y_cycle = []
x_star = []
y_star = []

#填充训练数据
for line in train_list:
    str_list = line.strip('\n').strip('\r').split('\t')
#     print(str_list)
    if str_list[2] == 'above' :
        x_cycle.append(float(str_list[0]))
        y_cycle.append(float(str_list[1]))
    else :
        x_star.append(float(str_list[0]))
        y_star.append(float(str_list[1]))


In [3]:

#x_cycle = []
#y_cycle = []
def my_reader_interface():
    def reader():
        label = ''
        for n in xrange(len(x_cycle)):
#             if n < 500:
#                 label = np.array(['above'])
#             else:
#                 label = np.array(['below'])
#             point = np.array([x_cycle[n], y_cycle[n]])
#             temp = (point, label)
            
            temp = np.array([x_cycle[n]]), np.array([y_cycle[n]])
            yield temp
    return reader


# '''
# readerTemp = my_reader_interface(x_cycle, y_cycle)
# for i in readerTemp():
#     print(i)
# '''



### 模型配置

线性回归的模型其实就是一个采用线性激活函数（linear activation，`LinearActivation`）的全连接层（fully-connected layer，`fc_layer`）：



In [4]:
paddle.init(use_gpu=False, trainer_count=1)
x = paddle.layer.data(name='x', type=paddle.data_type.dense_vector(1))
y_predict = paddle.layer.fc(input=x,
                                size=1,
                                act=paddle.activation.Linear())
y = paddle.layer.data(name='y', type=paddle.data_type.dense_vector(1))
cost = paddle.layer.mse_cost(input=y_predict, label=y)



### 保存网络拓扑



In [5]:
# Save the inference topology to protobuf.
# inference_topology = paddle.topology.Topology(layers=y_predict)
# with open("inference_topology.pkl", 'wb') as f:
#     inference_topology.serialize_for_inference(f)



### 创建参数



In [6]:
parameters = paddle.parameters.create(cost)


[INFO 2017-11-29 07:56:54,293 networks.py:1482] The input order is [x, y]
[INFO 2017-11-29 07:56:54,295 networks.py:1488] The output order is [__mse_cost_0__]



### 创建Trainer



In [7]:
optimizer = paddle.optimizer.Momentum(momentum=0)

trainer = paddle.trainer.SGD(cost=cost,
                             parameters=parameters,
                             update_equation=optimizer)


[INFO 2017-11-29 07:56:56,085 networks.py:1482] The input order is [x, y]
[INFO 2017-11-29 07:56:56,095 networks.py:1488] The output order is [__mse_cost_0__]



### 读取数据且打印训练的中间信息

PaddlePaddle提供一个
[reader机制](https://github.com/PaddlePaddle/Paddle/tree/develop/doc/design/reader)
来读取数据。 Reader返回的数据可以包括多列，我们需要一个Python dict把列
序号映射到网络里的数据层。



In [8]:
feeding={'x': 0, 'y': 1}
readerTemp = my_reader_interface()

print readerTemp

for r in readerTemp():
    print r

<function reader at 0x7f26adea68c0>
(array([ 3.672936]), array([ 737.759068]))
(array([ 63.876809]), array([-274.355426]))
(array([ 11.107609]), array([ 113.841277]))
(array([ 1.431797]), array([ 299.263575]))
(array([ 28.62928]), array([ 259.759192]))
(array([ 60.285056]), array([-107.37682]))
(array([ 81.613431]), array([ 396.635093]))
(array([ 58.656101]), array([ 11.604848]))
(array([ 11.071666]), array([ 647.152796]))
(array([ 15.652827]), array([ 614.173597]))
(array([ 15.489736]), array([ 562.664771]))
(array([ 41.050511]), array([-87.608473]))
(array([ 68.704425]), array([ 323.249695]))
(array([ 29.657592]), array([-56.364298]))
(array([ 88.444092]), array([ 210.522823]))
(array([ 87.539924]), array([ 93.998184]))
(array([ 58.123564]), array([ 183.81478]))
(array([ 59.188688]), array([-51.17026]))
(array([ 35.689108]), array([ 278.969059]))
(array([ 51.859443]), array([-49.784498]))
(array([ 38.687686]), array([ 316.038963]))
(array([ 83.417851]), array([ 295.380928]))
(array([


此外，我们还可以提供一个 event handler，来打印训练的进度：




### 开始训练



In [ ]:

readerTemp = my_reader_interface()

print readerTemp

myReader = paddle.reader.shuffle(
    readerTemp, 
    buf_size=20)

myBatch = paddle.batch(
        myReader,
        batch_size=2)

trainer.train(
    reader=myBatch,
    feeding=feeding,    
    num_passes=30)

In [ ]:
test_data_creator = paddle.dataset.uci_housing.test()
test_data = []
test_label = []

for item in test_data_creator():
    test_data.append((item[0],))
    test_label.append(item[1])
    if len(test_data) == 5:
        break



#### 2. 推测 inference



In [ ]:
# load parameters from tar file.
# users can remove the comments and change the model name
# with open('params_pass_20.tar', 'r') as f:
#     parameters = paddle.parameters.Parameters.from_tar(f)

probs = paddle.infer(
    output_layer=y_predict, parameters=parameters, input=test_data)

for i in xrange(len(probs)):
    print "label=" + str(test_label[i][0]) + ", predict=" + str(probs[i][0])
